In [2]:
#Q1_2

In [3]:
import urllib.request
import urllib.parse

from bs4 import BeautifulSoup

def save_html_to_file(html, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(html)


In [4]:
def extract_press_releases(seed_url, target_keyword, min_num_urls):
    
    child_urls = []
    seen_url=[]
    result_urls = []
    page_num = 0
    press_num = 0
    
    while press_num < min_num_urls and len(child_urls)>=0:
        
        if len(child_urls) == 0: #find all child urls in current page
            
            #print('page_num = '+ str(page_num))
            page = seed_url + '/page/{}'
            page_url = page.format(page_num)
            req = urllib.request.Request(page_url, headers={'User-Agent': 'Mozilla/5.0'})
            webpage = urllib.request.urlopen(req).read()
            soup = BeautifulSoup(webpage, features='html.parser')

            for a_tag in soup.find_all('a', href=True):
            
                org_child_url = a_tag.get('href')
                child_url = urllib.parse.urljoin(page_url, org_child_url)
            
                if child_url not in seen_url and seed_url in child_url:
                
                    seen_url.append(child_url)
                    child_urls.append(child_url)
            
            #print('Number of child urls in page'+ str(page_num) +" : "+ str(len(child_urls)))
            page_num += 1
        
        try:
            current_url = child_urls.pop(0)
            req = urllib.request.Request(current_url, headers={'User-Agent': 'Mozilla/5.0'})
            webpage = urllib.request.urlopen(req).read()
            soup = BeautifulSoup(webpage, features='html.parser')
            tag = soup.find('span', class_="ep_name", text='Plenary session')
            #print('left child urls ='+str(len(child_urls)) )
            
            if tag:
                text = soup.get_text()
                if 'crisis' in text.lower():
                    press_num += 1
                    #print('press_num = '+ str(press_num))
                    #print(current_url)
                    result_urls.append(current_url)
                    save_html_to_file(str(soup), f'2_{press_num}.txt')
            #else:
                #print('None')
        except:
            continue
            
    return result_urls


            


In [ ]:
seed_url = "https://www.europarl.europa.eu/news/en/press-room"
target_keyword = "crisis"
min_num_urls = 10

press_releases = extract_press_releases(seed_url, target_keyword, min_num_urls)
